In [ ]:
""" 
Author: maxwell.cook@colorado.edu
"""

import os, sys, gc, time
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

from scipy.stats import pearsonr

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

In [ ]:
fp = os.path.join(projdir,'data/tabular/mod/gridstats_lf_gs.csv')
tm = pd.read_csv(fp)
tm.drop(columns=['Unnamed: 0'], inplace=True)
tm.columns

In [ ]:
len(tm)

In [ ]:
# load the CBI data
fp = os.path.join(projdir,'data/earth-engine/exports/gridstats_cbibc_forest.csv')
sev = pd.read_csv(fp)
sev.drop(columns=['system:index','.geo'], inplace=True)
# create a unique ID
sev['grid_idx'] = sev['Fire_ID'].astype(str) + sev['grid_index'].astype(str)
print(sev.columns)

# Rename the columns for clarity
cols = ['mean', 'stdDev', 'p90', 'p95', 'p97', 'p99']
for col in cols:
    new_name = 'CBIbc_' + col
    sev.rename(columns={col: new_name}, inplace=True)
sev.drop(columns=['Fire_ID','grid_index'], inplace=True)
sev['grid_idx'] = sev['grid_idx'].astype(int)
# merge the dataframes
tm_sev = tm.merge(sev, on='grid_idx', how='left')
tm_sev[['grid_idx','fortypnm_gp','proportion','frp_max','frp_csum',
        'CBIbc_mean','CBIbc_p90','CBIbc_p95']].head(10)

In [ ]:
# Plot FRP by dominant forest type
# # create a dataframe with dominance (>50% of the same species)
# dom = tm_sev[tm_sev['fortyp_pct'] > 0.50]
# Find the median and sort the forest types by FRP
med = tm_sev.groupby('fortypnm_gp')['CBIbc_p90'].median().sort_values(ascending=False)
ordered = med.index.tolist()

# plot it
plt.figure(figsize=(6, 4)) 

sns.boxplot(
    data=tm_sev, 
    x='CBIbc_p90', 
    y='fortypnm_gp', 
    order=ordered, 
    palette='inferno', 
    dodge=False)

# plt.xscale('log') # log-scale FRP
plt.xlabel('Composite Burn Index')
plt.ylabel('Forest Type')
plt.tight_layout()

out_plot = os.path.join(projdir, 'figures/Figure_LF_Dom_CBIbc90.png')
plt.savefig(out_plot, dpi=300, bbox_inches='tight')

plt.show()
print(f"\nSaved to: {out_plot}\n")

In [ ]:
tm_sev.columns

In [ ]:
print(len(tm))
print(len(sev))
print(len(tm_sev))

In [ ]:
print(f"\nNaNs in CBI: {tm_sev['CBIbc_p90'].isna().sum()}\n")
print(f"NaNs in FRP: {tm_sev['frp_max_day'].isna().sum()}\n")
print(f"NaNs in FORTYP: {tm_sev['fortypnm_gp'].isna().sum()}\n")

In [ ]:
tm_sev = tm_sev[~tm_sev['CBIbc_p90'].isna()]
print(len(tm_sev))

In [ ]:
print(tm_sev.columns)

In [ ]:
tm_sev['fortypnm_gp'].unique()

In [ ]:
# save the table out
out_fp = os.path.join(projdir,'data/tabular/mod/gridstats_lf_gs_frp-cbi.csv')
tm_sev.to_csv(out_fp)
print(f"Saved to: {out_fp}")

In [ ]:
gc.collect()